In [1]:
import numpy as np

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

import torchvision

from tqdm.notebook import tqdm

import train as training

In [2]:
dire = "./Data/NvP" # directory of dataset
# loading data loader
trainloader, testloader = training.preprocess_data(directory=dire, batch_size=64, test_size=0.3, rand_num=40, worker=2)

{0: 426, 1: 426}


In [3]:
# load pretrained model
# vgg16 = torchvision.models.vgg16(pretrained=True)
# googlenet = torchvision.models.googlenet(pretrained=True)
# efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [4]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained: nn.Module):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(2048, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, 2),
                                         nn.Softmax()
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [5]:
# define params here
net = ClassifierNet(resnet50)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9)

criterion = nn.CrossEntropyLoss()
epochs = 20
training.run(net, optimizer, scheduler, criterion, epochs, trainloader, testloader)

AttributeError: module 'train' has no attribute 'train'